### Import data properly

I have a dataset for semantic segmentation saved locally on my computer. Specifically, i have tif image pairs, where one image is the RGB image, and the label is a binary mask saved as a tif. I want to finetune a huggingface segformer using this data. I have never finetuned a huggingface model before, and don't know in what format my dataset needs to be. Write a description of the format of the dataset that the huggingface standard segformer expects. Subsequently, write the method (including any code if needed) that i should use to obtain this format from my tif image pairs. 


Folder structure will be: 

Dataset 
    |__RGB
    |__HS
    |__DEM
    |__Annotation
    |__labels.csv

#### Define dataset 

In [ ]:
from transformers import SegformerImageProcessor
import pandas as pd 
from torch.utils.data import Dataset
import os
from PIL import Image

# adapted from https://github.com/NielsRogge/Transformers-Tutorials/blob/master/SegFormer/Fine_tune_SegFormer_on_custom_dataset.ipynb
class SemanticSegmentationDataset(Dataset):
    """Image (semantic) segmentation dataset."""

    def __init__(self, root_dir, labels_df, image_processor):
        """
        Args:
            root_dir (string): Root directory of the dataset containing the images + annotations.
            image_processor (SegFormerImageProcessor): image processor to prepare images + segmentation maps.
            labels_df: pd.Dataframe of the image names to be used in the dataset
        """
        self.root_dir = root_dir
        self.image_processor = image_processor
        self.labels_df = labels_df

        self.img_dir = os.path.join(self.root_dir, "rgb")
        self.ann_dir = os.path.join(self.root_dir, "annotations")

    def __len__(self):
        return len(self.labels_df)

    def __getitem__(self, idx):
        img_name = self.labels_df['filenames'][idx]
        img_path = os.path.join(self.img_dir, f"{img_name}.tif")
        ann_path = os.path.join(self.ann_dir, f"{img_name}.tif")

        image = Image.open(img_path)
        segmentation_map = Image.open(ann_path)

        # randomly crop + pad both image and segmentation map to same size
        encoded_inputs = self.image_processor(image, segmentation_map, return_tensors="pt")

        for k,v in encoded_inputs.items():
          encoded_inputs[k].squeeze_() # remove batch dimension

        return encoded_inputs
    

"""
WARNING: 
by default the image processor below will resize the image (to 512*512).
Essentially i don't want this, HOWEVER it might be necessary to be able
to use the weights from pretraining. TODO: Find out whether that's so.
"""

# split test and train set 
labels_file = "/Users/nadja/Documents/UU/Thesis/Data/TINYsampleFINAL/tinydataset/palsa_labels.csv"
all_imgs = pd.read_csv(labels_file, usecols=[0], header=0, names=['filenames'])
train_labels = all_imgs.sample(frac=0.8)
val_labels = all_imgs.drop(train_labels.index)

root_dir = "/Users/nadja/Documents/UU/Thesis/Data/TINYsampleFINAL/tinydataset"
image_processor = SegformerImageProcessor(
    img_mean = [74.90, 85.26, 80.06], # use mean calculated over our dataset
    img_std = [15.05, 13.88, 12.01], # use std calculated over our dataset
    do_reduce_labels=True
    # additionally, do i want some augmentation? 
    )

train_dataset = SemanticSegmentationDataset(root_dir, train_labels, image_processor)
val_dataset = SemanticSegmentationDataset(root_dir, val_labels, image_processor)

In [1]:
from PIL import Image 

img = Image.open("/Users/nadja/Documents/UU/Thesis/Data/TINYsampleFINAL/tinydataset/rgb/732_52_5000_2015_crop_88.tif")

In [2]:
img.show()

In [13]:
import pandas as pd
labels_file = "/Users/nadja/Documents/UU/Thesis/Data/TINYsampleFINAL/tinydataset/palsa_labels.csv"
# all_imgs = pd.read_csv(labels_file, index_col=0)

all_imgs = pd.read_csv(labels_file, usecols=[0], header=0, names=['filenames'])

In [22]:
all_imgs

,filenames
0,732_52_5000_2015_crop_88
1,732_52_5000_2015_crop_88_aug


In [23]:
train = all_imgs.sample(n=1)

In [24]:
train

,filenames
0,732_52_5000_2015_crop_88


In [25]:
all_imgs.drop(train.index)

,filenames
1,732_52_5000_2015_crop_88_aug


In [20]:
all_imgs2 = pd.read_csv(labels_file, index_col=0)


In [21]:
all_imgs2

,palsa_percentage
732_52_5000_2015_crop_88,0
732_52_5000_2015_crop_88_aug,0
